    HOT WIRE ANEMOMETER AQUISITON Z v.3
    LEA - Wind Tunnel
    python version

Developer: Rafael C. F. Mendes

Date: 04/10/2023   (v1: 24/11/2016)

Description: Use this program to move hot wire probe and taking data from CTA module.
             
Hardware: 
   - Arduino Uno (probe positioning system) with GRBL library (see more at https://github.com/grbl/grbl)
   - National Instrument - USB-4332 **Warning: Working voltage -10 to 10V**                 
   - Hot wire anemometer - miniCTA DANTEC

Before run it, install the following libraries:

    - pip install nidaqmx
    - pip install pyserial

In [1]:
#libraries

import nidaqmx          
import time
import numpy as np
import serial           
from datetime import datetime
from nidaqmx.constants import LineGrouping
from nidaqmx.constants import Edge
from nidaqmx.constants import AcquisitionType 
import os
from scipy.optimize import curve_fit


In [2]:
# Configure your experiment here!
# Configure o seu experimento aqui!

#Sampling Definitions 
SampleRate=1000
AquisitionTimeInSeconds=5


y = [310] # y coordinate to measure the profile; y=310 ==> middle 
x = [1000]


dz=10 # delta Y in mm

# Delay between the points
DeltaT = 1 # delay time in seconds

# Define the serial port and baud rate
port = 'COM3'  # Update with your Arduino's port
baud_rate = 115200

In [3]:
# Create coordinates vectors
z = np.arange(20, 351, dz)
Ysize, = np.shape(y)
Zsize, = np.shape(z)



In [4]:
# Function to call NI system 

def acquire_data(SampleRate, AquisitionTimeInSeconds):
    with nidaqmx.Task() as master_task:
        Sample=SampleRate*AquisitionTimeInSeconds
        master = master_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        master_task.timing.cfg_samp_clk_timing(SampleRate, source='', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)
        master_task.stop()
        master_data = master_task.read(Sample, timeout=AquisitionTimeInSeconds+1.0)
    return master_data


In [5]:
# Create a serial object
ser = serial.Serial(port, baud_rate)

# Check if the serial port is open
if ser.is_open:
    print(f"Serial port {port} is open. Ready to communicate.")

ser.reset_input_buffer()
ser.reset_output_buffer()
ser.readline()

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Close the serial port
#ser.close()

# Send a command to Arduino
command = '$H\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it



Serial port COM3 is open. Ready to communicate.


Response from Arduino: Grbl 1.1f ['$' for help]
Response from Arduino: [MSG:'$H'|'$X' to unlock]


4

In [6]:

folder_name = "Velocity data"

if not os.path.exists(folder_name):
    # Create the folder
    os.mkdir(folder_name)

now = datetime.now()
date_string = now.strftime('%d-%b-%Y')
time_string = now.strftime('%H-%M-%S')

folder_name = os.path.join(folder_name,f"{date_string}_{time_string}")
os.mkdir(folder_name)


In [7]:
i = 0

command = f"y-{y[i]} z-0 \r\n" 
print(command)
ser.write(command.encode())  # Encode the command and send it
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)
time.sleep(20)  # Sleep for 20 seconds

y-310 z-0 



Response from Arduino: ok


In [8]:
i=0
command = f"y-{y[0]}\r\n"
ser.write(command.encode())
print(command)
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)
time.sleep(DeltaT)  # Sleep DeltaT

command = f"x-{x[0]}\r\n"

ser.write(command.encode())
print(command)
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)
time.sleep(DeltaT)  # Sleep DeltaT

for j in range(Zsize):

    command = f"z-{z[j]}\r\n"
   # ser.write(command.encode())  # Encode the command and send it
   # response = ser.readline().decode().strip()  # Read the response from Arduino
   # print("Response from Arduino:", response)
   # ser.write(command.encode())  # Encode the command and send it
   # response = ser.readline().decode().strip()  # Read the response from Arduino
   # print("Response from Arduino:", response)    

        
    filename = os.path.join(folder_name, f"{x[i]}x{y[i]}x{z[j]}.txt")



    with open(filename, 'w') as s:
       
        print(command)
        ser.write(command.encode())  # Encode the command and send it
        response = ser.readline().decode().strip()  # Read the response from Arduino
        print("Response from Arduino:", response)    
    time.sleep(DeltaT)  # Sleep DeltaT
    
    data=acquire_data(SampleRate, AquisitionTimeInSeconds)

    data_array = np.array(data)

   # SizeRowData, SizeColunData=np.shape(data)

    SizeRowData, = data_array.shape  # Note the comma after SizeRowData
         


    with open(filename, 'wt') as fid:
        for value in data:
            fid.write(f'{value:12.8f}\n')





# Send a command to Arduino
command = '$H\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it

ser.close()

y-310

Response from Arduino: ok
x-1000

Response from Arduino: ok
z-20

Response from Arduino: ok
z-30

Response from Arduino: ok
z-40

Response from Arduino: ok
z-50

Response from Arduino: ok
z-60

Response from Arduino: ok
z-70

Response from Arduino: ok
z-80

Response from Arduino: ok
z-90

Response from Arduino: ok
z-100

Response from Arduino: ok
z-110

Response from Arduino: ok
z-120

Response from Arduino: ok
z-130

Response from Arduino: ok
z-140

Response from Arduino: ok
z-150

Response from Arduino: ok
z-160

Response from Arduino: ok
z-170

Response from Arduino: ok
z-180

Response from Arduino: ok
z-190

Response from Arduino: ok
z-200

Response from Arduino: ok
z-210

Response from Arduino: ok
z-220

Response from Arduino: ok
z-230

Response from Arduino: ok
z-240

Response from Arduino: ok
z-250

Response from Arduino: ok
z-260

Response from Arduino: ok
z-270

Response from Arduino: ok
z-280

Response from Arduino: ok
z-290

Response from Arduino: ok
z-300

Response f